**Getting all url for single anime page access**

In [58]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np
ur=[]
for i in range(1,300):
    url='https://www.anime-planet.com/anime/all?page={}'.format(i)
    headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
            }
    response=requests.get(url,headers=headers)
    con=response.content
    soup=BeautifulSoup(con ,'lxml')
    for name in soup.find_all('li',class_="card"):
        for na in name.find_all('a'):
            ur.append(na.get('href'))

KeyboardInterrupt: 

**creating dataframe**

In [50]:
col_names=['url',"Name",'Japanese_name','Type','Episodes','Studio','Release_season','Tags','Rating','Release_year','End_year','Description','Content_Warning','Related_Mange','Related_anime','Voice_actors','staff','Rank']
df=pd.DataFrame(columns=col_names)
df['url']=ur

**getting all info for each anime listed on site**

In [52]:
i=0
for u in ur:
    url="https://www.anime-planet.com{}".format(u)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
        }
    response=requests.get(url,headers=headers)
    con=response.content
    soup=BeautifulSoup(con ,'lxml')
    
    print(i)
    print(u)
    
    #names
    eng_name=[]
    if soup.find_all(itemprop='name'):
        for en in soup.find_all(itemprop='name'):
            if en!=None:
                eng_name.append(en.text.strip())
        df.iloc[i,1]=eng_name[0]
    else:
        df.iloc[i,1]=np.nan

    jap_name=[]
    if soup.find(class_='aka'):
        for name in soup.find(class_='aka'):
            jap_name.append(name.text.strip())
        df.iloc[i,2]=jap_name[0]
    else:
        jap_name=np.nan

    
    
    
    #type
    if soup.find(class_='type'):
        types=soup.find('span',class_='type').text
        types = re.sub(r'[\n\(\)]', '     ', types)
        type_=types[:5]

        df.iloc[i,3]=type_
        if type_=='Movie':
            df.iloc[i,4]=types[15:29]
        else:
            df.iloc[i,4]=types[11:15]
    else:
        df.iloc[i,4]=np.nan
        

    #studio   
    studio=[]
    
    for ele in soup.find_all(class_='pure-1 md-1-5'):
        #if ele.find('a'):
        a=ele.find('a')
        if a!=None:
            studio.append(a.text)
    
            df.iloc[i,5]=studio[0]
    #print(studio)
    #release season 
    if len(studio)==1:
        df.iloc[i,6]=np.nan
    if len(studio)==2:
        if studio[1][:4]=='Fall':
            df.iloc[i,6]=studio[1][:5]
        else:
            df.iloc[i,6]=studio[1][:6]
    #tags
    tags=[]
    for tag in soup.find_all(class_='pure-1 md-3-5'):
        if tag.find_all('ul'):
            for ul in tag.find_all('ul'):
                for li in ul.find_all('li'):
                    tags.append(li.text.strip())

            tags = ', '.join(tags)
            df.iloc[i,7]=tags
        else:
            df.iloc[i,7]=np.nan
                
    #rating
    if soup.find_all(class_='avgRating'):
        rating=soup.find(class_='avgRating').text.strip()
        rating=rating[:4]
        df.iloc[i,8]=rating
    else:
        df.iloc[i,8]=np.nan
    
    #release end year
    year=[]
    if soup.find(class_='iconYear'):
        for time in soup.find(class_='iconYear'):
            year.append(time.text.strip())

        release_year=year[0][:5]

        end_year=year[0][7:]

        df.iloc[i,9]=release_year
        df.iloc[i,10]=end_year
    else:
        df.iloc[i,9]=np.nan
        df.iloc[i,10]=np.nan
    
    #synopsis
    description=[]
    for desc in soup.find_all(class_='pure-1 md-3-5'):
        if desc.find('p'):
            description.append(desc.find('p').text.strip())
            df.iloc[i,11]=description[0]
        else:
            df.iloc[i,11]=np.nan
    
    #content warning
    warn=[]
    if soup.find_all(class_='tags tags--plain'):
        for tag in soup.find_all(class_='tags tags--plain'):
            for ul in tag.find_all('ul'):
                for li in ul.find_all('li'):
                    warn.append(li.text.strip())
        warn = ', '.join(warn)
    else:
        warn=np.nan


    df.iloc[i,12]=warn
    #releated manga
    manga=[]
    if soup.find_all(id='tabs--relations--manga--same_franchise'):
        for man in soup.find_all(id='tabs--relations--manga--same_franchise'):
            for mang in man.find_all(class_='RelatedEntry__name rounded-card__title'):
                manga.append(mang.text.strip())
        manga = ', '.join(manga)
        df.iloc[i,13]=manga
    else:
        df.iloc[i,13]=np.nan

    
    
    #releated anime
    anime=[]
    if soup.find_all(id='tabs--relations--anime--same_franchise'):
        for an in soup.find_all(id='tabs--relations--anime--same_franchise'):
            for mang in an.find_all(class_='RelatedEntry__name rounded-card__title'):
                anime.append(mang.text.strip())

        anime = ', '.join(anime)
        df.iloc[i,14]=anime
    else:
        df.iloc[i,14]=np.nan
    
    #voic actors
    voice=[]
    if soup.find_all(class_='CharacterCard__main'):
        for actor in soup.find_all(class_='CharacterCard__main'):
            for y in actor.find_all(class_='CharacterCard__content'):
                voice.append(y.text.title().strip())

        voice = ', '.join(voice)
        voice = re.sub(r'(\n \n)', ' : ', voice)
        df.iloc[i,15]=voice
    else:
        df.iloc[i,15]=np.nan
    
    #staff
    staff=[]
    if soup.find_all(class_='EntryPage__content__section EntryPage__content__section__staff castaff'):
        for desc in soup.find_all(class_='EntryPage__content__section EntryPage__content__section__staff castaff'):
            for y in desc.find_all(class_='CharacterCard__content'):
                staff.append(y.text.title().strip())

        staff = ', '.join(staff)
        staff = re.sub(r'[\n]', ' : ', staff)
        df.iloc[i,16]=staff
    else:
        df.iloc[i,16]=np.nan
    
    #rank
    ranks=[]
    for rank in soup.find_all(class_='pure-1 md-1-5')[4]:
        if rank!=None:
            ranks.append(rank.text.strip())
            df.iloc[i,17]=ranks[0][6:]
        else:
            df.iloc[i,17]=np.nan
        
    i=i+1


0
/anime/oster-project-ookami-nanka-kowakunai
1
/anime/the-idolmaster-cinderella-girls-theater
2
/anime/kero-kero-keroppi-no-gulliver-no-bouken
3
/anime/umi-no-yami-tsuki-no-kage
4
/anime/asian-kung-fu-generation-atarashii-sekai
5
/anime/sekai-meisaku-douwa-oyayubi-hime
6
/anime/inugami-san-to-nekoyama-san-special
7
/anime/shin-koihime-musou-otome-tairan-ova-omake
8
/anime/violinist-of-hamelin-movie
9
/anime/lupin-the-3rd-pilot-film
10
/anime/the-qwaser-of-stigmata-portrait-of-the-empress
11
/anime/yume-miru-jump
12
/anime/beniimo-musume
13
/anime/umbrella-fighter
14
/anime/irodorimidori-ok-alright
15
/anime/inishie-no-megami-to-houseki-no-ite
16
/anime/watashitachi-luck-logic-bu-2
17
/anime/bakumatsu-crisis
18
/anime/night-warriors-darkstalkers-revenge
19
/anime/duel-masters-victory
20
/anime/paris-no-isabelle
21
/anime/deep-sea-tentacle
22
/anime/tenka-muteki-no-gouyaaman
23
/anime/bakegyamon
24
/anime/stratos-4-ova-stratos-41-dutch-roll
25
/anime/a-sisters-all-you-need-specials
26
/

228
/anime/kaitou-tenshi-twin-angel-kyun-kyun-tokimeki-paradise-ova
229
/anime/rerulili-special-girl
230
/anime/wanpaku-oomukashi-kum-kum
231
/anime/kano-rere
232
/anime/the-new-adventures-of-maya-the-honeybee
233
/anime/hachioji-p-blue-star
234
/anime/last-hope
235
/anime/sister-princess
236
/anime/oster-project-on-the-rocks
237
/anime/knight-hunters-eternity
238
/anime/yowai-robot-to-10-no-story-project
239
/anime/smile-down-the-runway-chiyukis-fashion-check
240
/anime/patta-potta-monta
241
/anime/x-men
242
/anime/mikasa-dai-senpai-to-manabu-anime-azur-lane
243
/anime/arigatou-2010
244
/anime/chiisai-aki-mitsuketa-1982
245
/anime/pokemon-pikachus-great-sparking-search
246
/anime/tsukipro-the-animation
247
/anime/wasurerareta-ningyou
248
/anime/tama-pawns
249
/anime/nejishiki-ash-na-milk-tea
250
/anime/printer-rec-music
251
/anime/mainichi-ga-nichiyoubi
252
/anime/of-mice-and-clockworks
253
/anime/rays-moonflowers
254
/anime/casshan-robot-hunter
255
/anime/a-centaurs-life
256
/anime/t

469
/anime/laidbackers
470
/anime/nejishiki-kimi-ga-someta-natsuzora
471
/anime/tekkaman-the-space-knight
472
/anime/chuumon-no-ooi-ryouriten-1958
473
/anime/nem-nemurihime
474
/anime/isekai-cheat-magician
475
/anime/vividred-operation
476
/anime/orangestar-n-buna-star-night-snow
477
/anime/transformers-zone
478
/anime/gundam-build-divers-prologue
479
/anime/seiren
480
/anime/candidate-for-goddess
481
/anime/akame-ga-kill-theater
482
/anime/universe
483
/anime/suteki-tantei-labyrinth
484
/anime/koi-no-sube-sube-manjuu-gani
485
/anime/taberu-hitotachi
486
/anime/my-boy
487
/anime/inaka-nezumi-to-machi-nezumi
488
/anime/hei-mao-jing-zhang
489
/anime/sokkuri-house
490
/anime/kageyama-tamio-no-double-fantasy
491
/anime/fukusuke
492
/anime/nemure-omoi-ko-sora-no-shitone-ni
493
/anime/peach-boy-riverside-mini-anime
494
/anime/gebaude-baude
495
/anime/rif-tsuyoku
496
/anime/tekkaman-blade-nagaki-tatakai-no-jokyoku
497
/anime/fake-type-fake-style-ii
498
/anime/pinocchio-p-medicine-for-good-kid

705
/anime/girly-air-force
706
/anime/punchnello-and-yerin-baek-nagseo
707
/anime/tayutayu-pure-my-heart
708
/anime/tactical-roar
709
/anime/kirin-the-noop
710
/anime/reset
711
/anime/winner
712
/anime/yoshida-katsuko-no-yabai-wa-sdgs-araburu-trouble-sustainable
713
/anime/steel-angel-kurumi-2
714
/anime/heidi-pilot
715
/anime/cinderella-express
716
/anime/ultra-dino-makera
717
/anime/island
718
/anime/mahou-no-mako-chan
719
/anime/human-crossing
720
/anime/fact-rise
721
/anime/fairy-ngoo
722
/anime/nem-a-sweet-typhoon
723
/anime/morinaga-ramune-mcshake-cm
724
/anime/the-town-musicians-of-bremen
725
/anime/spirit-warrior
726
/anime/the-diary-of-tortov-roddle-specials
727
/anime/chibi-devi
728
/anime/bokura-machi-bouzu
729
/anime/deaerutte-ii-ne
730
/anime/kuchiroro-umi
731
/anime/mameshiba-no-ouchi-de-tanoshimu-mame-chishiki
732
/anime/white-album
733
/anime/classroom-crisis-special
734
/anime/chinzei-hachirou-tametomo
735
/anime/akai-koudan-zillion
736
/anime/salsa-ii-na-ii-ne
737
/an

952
/anime/nem-parallel-world
953
/anime/goku-ii-midnight-eye
954
/anime/channel-55
955
/anime/recently-my-sister-is-unusual-ova
956
/anime/memories-off-3-5
957
/anime/mah-dead-end-street
958
/anime/kingdom-of-chaos-born-to-kill
959
/anime/sugai-kun-to-kazoku-ishi-x-blue-note-tokyo-hen
960
/anime/queens-blade-ova
961
/anime/new-york-trip
962
/anime/breakers
963
/anime/machine-robo-revenge-of-chronos
964
/anime/minami-kitasono-yuu-kiri
965
/anime/double-hard
966
/anime/pokemon-gotta-dance
967
/anime/code-e
968
/anime/ladies-versus-butlers-music-clip
969
/anime/23-exe-ochatime
970
/anime/rozen-maiden-detective-kun-kun
971
/anime/yasuyuki-okamura-viva-namida
972
/anime/line-town
973
/anime/sagashita-no-wa-oheya-mitsuketa-no-wa-mirai
974
/anime/yubisaki-kara-honki-no-netsujou-2-koibito-wa-shouboushi
975
/anime/kamen-rider-sd
976
/anime/devil-hunter-yohko
977
/anime/yosuga-no-sora-in-solitude-where-we-are-least-alone
978
/anime/chronicles-of-the-going-home-club
979
/anime/eiyuu-ni-wa-narena

...

7051
/anime/snack-time
7052
/anime/sleepless-night
7053
/anime/zhan-yi-yingxiong
7054
/anime/chikyuu-wa-guruguru-merry-go-round
7055
/anime/hua-jianghu-zhi-huan-shi-men-sheng-xu-pian
7056
/anime/xiyou-ji-wu-jian-baobei
7057
/anime/dahong-ying
7058
/anime/zi-zuo-zi-shou
7059
/anime/qi-qi-piao-youji
7060
/anime/cha-bu-duo
7061
/anime/bing-shang-yu-xian
7062
/anime/spy-x-family
7063
/anime/kami-no-ue-no-pilot
7064
/anime/merry-go-round
7065
/anime/kachikachi-yama
7066
/anime/issunboushi
7067
/anime/neko-to-nezumi
7068
/anime/komi-cant-communicate-2nd-season
7069
/anime/marie-ishikawa-love-sayonara-wa-iwanai
7070
/anime/the-zero-century-emeraldas
7071
/anime/the-zero-century-herlock
7072
/anime/the-zero-century-maetel
7073
/anime/sore-ike-anpanman-roll-to-laura-ukigumojou-no-himitsu
7074
/anime/moshi-jue-xing-zhi-ruqin
7075
/anime/moshi-juexing-zhi-ruqin-2nd-season
7076
/anime/bousou-sengokushi
7077
/anime/chao-shen-xueyuan-zhi-hei-jia
7078
/anime/madoromi-no-shuumatsu
7079
/anime/nono-zu


7280
/anime/mebae-no-ashita
7281
/anime/hibike-wadaiko
7282
/anime/dokkun-dokkun
7283
/anime/jingju-mao-3-zhi-cheng-feng-polang
7284
/anime/the-gift-of-the-magi
7285
/anime/cubic-centimeter
7286
/anime/canigou-tape
7287
/anime/niguang-zhi-chan
7288
/anime/huaguo-shan-zhuan
7289
/anime/kuaile-dongxi-2nd-season
7290
/anime/kuaile-dongxi-3rd-season
7291
/anime/love-live-nijigasaki-high-school-idol-club-season-2
7292
/anime/cang-feng
7293
/anime/lethe
7294
/anime/jie-jie-xiong-yu-di-di-niao
7295
/anime/amiina-sign
7296
/anime/shounen-sunday-cm-firefighter-daigo-of-fire-company-m
7297
/anime/shounen-sunday-cm-historys-strongest-disciple-kenichi
7298
/anime/baobao-yue-xuetang-xiha-maoxian-dao
7299
/anime/the-little-mermaid-attack-of-the-pirates
7300
/anime/the-dawn-of-ape
7301
/anime/alice-in-cyberland-dive-2
7302
/anime/space-shower-tv-big-hits-id-gupon
7303
/anime/colorful-taiyou-tour
7304
/anime/long-ya-xing
7305
/anime/pochi-hana
7306
/anime/hahajing-hua-yuan
7307
/anime/xiao-cai-feng
73

7513
/anime/disney-twisted-wonderland
7514
/anime/saezuru-doubutsu-wa-yarushikanai-specials
7515
/anime/dou-po-cangqiong-5th-season
7516
/anime/fuyu-no-niji
7517
/anime/good-theater-manners-from-kamen-rider-zero-one-and-kamen-rider-saber
7518
/anime/arknights-prelude-to-dawn
7519
/anime/yige-xin-zuqiu
7520
/anime/flow-of-life
7521
/anime/etude-number-5-in-e-minor
7522
/anime/pan-no-march
7523
/anime/kohaku-no-mahou
7524
/anime/koutsuu-anzen-inu-signal-no-oyako-nikki-tobidashi-chuui
7525
/anime/library-cross-infinite
7526
/anime/empress-chung
7527
/anime/dendou-sakusen-gattaiou
7528
/anime/saru-getchu-movie-ougon-no-pipo-helmet-ukki-battle
7529
/anime/yahabe
7530
/anime/nyamen
7531
/anime/konpeki-no-kantai-tokubetsu-hen-sourai-kaihatsu-monogatari
7532
/anime/tanuki
7533
/anime/taoqi-bao-ma-xiaotiao
7534
/anime/hare-butai
7535
/anime/mr-digital-tokoro
7536
/anime/daojian-shen-huang
7537
/anime/huo-xing-wa-maoxian-leyuan
7538
/anime/chengzi-gushi
7539
/anime/the-adventures-of-the-little-p

7755
/anime/moonlight-adventure
7756
/anime/gao-nuren-he-ai-zhangfu
7757
/anime/anning
7758
/anime/zhengzhi
7759
/anime/xiao-e-yu-hong-fangzi
7760
/anime/dont-toy-with-me-miss-nagatoro-2
7761
/anime/tiantian-jiayuan
7762
/anime/wai-xing-ren-ago
7763
/anime/wai-xing-ren-ago-4th-season
7764
/anime/tatsunoko-fight-op
7765
/anime/bu-cha-ban-fen-hao
7766
/anime/wangu-jian-shen
7767
/anime/liang-bu-yi-2nd-season
7768
/anime/odd-taxi-in-the-woods
7769
/anime/omoinotake-wakusei
7770
/anime/reizouko-no-tsukenosuke
7771
/anime/yama-no-kamisama
7772
/anime/mashin-mukashi-banashi-gekijou
7773
/anime/tokyo-gambo
7774
/anime/kiyakiya
7775
/anime/the-quintessential-quintuplets-movie
7776
/anime/yousei-dick
7777
/anime/fuwafuwa-hour-puipui-and-muumuu
7778
/anime/kijeu-csi-gwahaksusadae
7779
/anime/decorations
7780
/anime/jingjing-lai-ledao
7781
/anime/charon-plutos-lonesomeness
7782
/anime/cosmopolite
7783
/anime/jin-huli-he-fen-fenghuang
7784
/anime/yongzhe-xueyuan
7785
/anime/buonomo
7786
/anime/bao

7979
/anime/baise-shandian
7980
/anime/bridgestone-chase-your-dream-the-journey-of-kosuke-hagino
7981
/anime/goblin-slayer-2
7982
/anime/boris-free
7983
/anime/peleliu-rakuen-no-guernica
7984
/anime/tooriame
7985
/anime/zhandou-ba-geji-shanyao-de-richang
7986
/anime/better-back-then
7987
/anime/make-my-day
7988
/anime/shachiku-san-wa-youjo-yuurei-ni-iyasaretai
7989
/anime/nostalgia
7990
/anime/qin-shi-mingyue-canghai-hengliu-xiaomeng-specials
7991
/anime/yi-tang-juchang-sanguo-yanyi
7992
/anime/fenghuang-ji-xiang-yu-qingming-shanghe-tu
7993
/anime/chengshi-jiyi-wo-men-de-jieri
7994
/anime/heisei-inu-monogatari-bow-genshi-inu-monogatari-bow
